# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**  
* 並び順: 任意（`ORDER BY` を付けない）  
* `NOT IN` は NULL 罠のため回避  
* 判定は **ID 基準**、表示は仕様どおりの列名と順序  

## 1) 問題

* `同一の tiv_2015 を持つ保険契約者が 1 名以上存在し、かつ (lat, lon) の位置が他と重複しない契約者の 2016 年投資額 (tiv_2016) の合計を求め、**小数 2 桁に丸めて**出力せよ。`
* 入力テーブル例: `Insurance(pid, tiv_2015, tiv_2016, lat, lon)`

**Table: `Insurance`**

| Column Name | Type  | Description |
|-------------|-------|-------------|
| pid         | int   | primary key |
| tiv_2015    | float |             |
| tiv_2016    | float |             |
| lat         | float | NOT NULL    |
| lon         | float | NOT NULL    |

* 出力仕様: `tiv_2016`（1 列 1 行、**小数 2 桁**）

参考例（問題文の Example と同義）:

| tiv_2016 |
|----------|
| 45.00    |

## 2) 最適解（単一クエリ）

> 方針: **ウィンドウ関数**で (a) `tiv_2015` が重複するか、(b) `lat, lon` がユニークか を 1 パスで判定し、該当行のみを合計。

```sql
WITH win AS (
  SELECT
    pid,
    tiv_2015,
    tiv_2016,
    lat,
    lon,
    COUNT(*) OVER (PARTITION BY tiv_2015)    AS cnt_same_tiv2015,
    COUNT(*) OVER (PARTITION BY lat, lon)     AS cnt_same_city
  FROM Insurance
)
SELECT
  ROUND(SUM(tiv_2016), 2) AS tiv_2016
FROM win
WHERE cnt_same_tiv2015 > 1
  AND cnt_same_city = 1;

-- Runtime 527 ms
-- Beats 56.61%

```

* `COUNT() OVER (PARTITION BY tiv_2015)` が 2 以上 → **同額の tiv_2015 が他にもいる**  
* `COUNT() OVER (PARTITION BY lat, lon)` が 1 → **所在地の (lat, lon) がユニーク**  
* 条件を満たす行の `tiv_2016` を `SUM` して `ROUND(…, 2)`

## 3) 代替解

> ウィンドウが使えない／重い環境を想定し、**相関 `EXISTS`** と **否定の `NOT EXISTS`**（`NOT IN` は使わない）で判定。

```sql
SELECT ROUND(SUM(i.tiv_2016), 2) AS tiv_2016
FROM Insurance i
WHERE EXISTS (
  SELECT 1
  FROM Insurance s
  WHERE s.pid <> i.pid
    AND s.tiv_2015 = i.tiv_2015
)
AND NOT EXISTS (
  SELECT 1
  FROM Insurance c
  WHERE c.pid <> i.pid
    AND c.lat = i.lat
    AND c.lon = i.lon
);

-- Runtime 527 ms
-- Beats 56.61%

```

* 前半 `EXISTS` … **同じ `tiv_2015` を持つ別行の存在**
* 後半 `NOT EXISTS` … **同じ `(lat, lon)` を持つ別行が存在しない** → 位置がユニーク

## 4) 要点解説

* **方針**: まず必要列のみで判定量を計算（ウィンドウ or 相関）→ 条件抽出 → 合計 → 丸め。
* **NULL / 重複**:
  * 入力仕様より `lat`, `lon` は **NOT NULL** のため `(lat, lon)` のユニーク判定は素直に可能。
  * `NOT IN` は NULL で壊れるため、**`EXISTS` / `NOT EXISTS`** を推奨。
* **安定性**: 結果順は要求されないので `ORDER BY` は付けない（余計なソートを避ける）。

## 5) 計算量（概算）

* **ウィンドウ解**: パーティションごとに `COUNT` → 実装依存だが概ね **O(N log N)**（パーティション内の並び替えが入る場合）〜 **O(N)**（ソート不要のストリーミング最適化時）。
* **EXISTS 解**: インデックス（`tiv_2015`, `(lat, lon)`）があれば各相関の存在確認は平均 **O(1)** に近く、全体 **O(N)** 近似。インデックスが無い場合は最悪 **O(N^2)**。

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 テーブル Insurance]
  B["前処理 ウィンドウ/相関で判定量算出"]
  C[条件抽出 tiv_2015重複 かつ 位置ユニーク]
  D["集計 SUM(tiv_2016)"]
  E["丸め ROUND(…, 2)"]
  F[出力 列:tiv_2016]
  A --> B
  B --> C
  C --> D
  D --> E
  E --> F
```

# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**  
* 並び順: 任意（`ORDER BY` なし）  
* `NOT IN` は NULL 罠のため回避  
* 判定は **ID/キー基準**、表示は仕様どおりの列名と順序  

あなたの2解法（ウィンドウ関数版 / `EXISTS` 版）は正しく、読みやすさも◎です。ここでは **実行計画がスキャン効率を取りやすい形** へのリライトと、**インデックス設計** の観点でチューニング案を示します。

## 1) 問題

* `同一の tiv_2015 を持つ保険契約者が 1 名以上存在し、かつ (lat, lon) が他と重複しない契約者の 2016 年投資額 (tiv_2016) の合計を求める（小数 2 桁丸め）。`
* 入力テーブル: `Insurance(pid, tiv_2015, tiv_2016, lat, lon)`
* 出力仕様: 列は `tiv_2016` のみ（1 行 1 列、**ROUND(…,2)**）

## 2) 最適解（単一クエリ・グループ化半結合）

> **発想**: ウィンドウ関数や相関ではなく、**事前に条件キーをグループ化**して *該当キーのみ* を `JOIN` で半結合（セミジョイン）する。集計系に強い MySQL の実行計画に乗りやすく、**大規模データで安定して速い** ことが多いです。

```sql
SELECT ROUND(SUM(i.tiv_2016), 2) AS tiv_2016
FROM Insurance AS i
JOIN (
  SELECT tiv_2015
  FROM Insurance
  GROUP BY tiv_2015
  HAVING COUNT(*) > 1
) AS d USING (tiv_2015)
JOIN (
  SELECT lat, lon
  FROM Insurance
  GROUP BY lat, lon
  HAVING COUNT(*) = 1
) AS u USING (lat, lon);

-- Runtime 508 ms
-- Beats 70.32%

```

### ねらい
* `d` は **重複する `tiv_2015` 値の集合**、`u` は **一意な `(lat,lon)` の集合**。
* どちらも **小さな集合** になりやすく、`USING` 結合で **対象行だけ** を走査 → `SUM`。
* 適切なインデックス（後述）で **グループ化と結合がインデックス・オンリー** に近づきます。

## 3) ウィンドウ版の微調整

あなたのウィンドウ解は正しいです。MySQL では **異なるパーティションキーのウィンドウを複数計算** すると、場合によりソートやワークテーブルが複数回発生します。以下の2点で負荷を抑えられる場合があります。

1. **幅の削減**: ウィンドウ計算時に不要列を持ち回らない（`pid` は最終的に不要）。
2. **CTEの使い方**: 8.0 以降、CTE は状況によりインライン化・/・マテリアライズされます。CTEを使わず**派生表**にしてもよいです（プラン次第）。

```sql
SELECT ROUND(SUM(tiv_2016), 2) AS tiv_2016
FROM (
  SELECT
    tiv_2015,
    tiv_2016,
    lat,
    lon,
    COUNT(*) OVER (PARTITION BY tiv_2015) AS cnt_same_tiv2015,
    COUNT(*) OVER (PARTITION BY lat, lon) AS cnt_same_city
  FROM Insurance
) AS w
WHERE cnt_same_tiv2015 > 1
  AND cnt_same_city = 1;

-- Runtime 572 ms
-- Beats 32.98%

```

※ 実測はワークロード依存です。`EXPLAIN ANALYZE` で比較をお勧めします。

## 4) 代替解（EXISTS 版のインデックス活用）

相関版は **読みやすさが長所**。以下のインデックスを置くと、相関サブクエリのルックアップが効きやすくなります。

```sql
-- 片方キーに対する完全一致探索を速くする
CREATE INDEX ix_insurance_tiv2015 ON Insurance (tiv_2015);
CREATE INDEX ix_insurance_lat_lon ON Insurance (lat, lon);
```

これでそれぞれの `EXISTS` / `NOT EXISTS` が **インデックスレンジスキャン** になりやすく、全体のコストが下がります。大規模データなら、上記グループ化半結合（§2）の方が安定するケースも多いです。

## 5) インデックス指針（まとめ）

* **必須候補**:  
  * `INDEX(tiv_2015)` … 重複検出（`GROUP BY` / `EXISTS`）
  * `INDEX(lat, lon)` … 位置のユニーク判定（`GROUP BY` / `NOT EXISTS` / `USING (lat,lon)`）
* **やりすぎ注意**: カバリング狙いで `(tiv_2015, lat, lon, tiv_2016)` の巨大複合は更新コストとストレージ増を招きやすい。まずは **上記2本** で十分なことが多いです。

## 6) 実行計画チェックリスト

* `EXPLAIN ANALYZE` で  
  * `JOIN` が **Using index / ref** で入っているか  
  * `GROUP BY` が **Using index for group-by** を活かせているか  
  * `Using temporary; Using filesort` が過多でないか  
* 結果順の要件は無いので **`ORDER BY` を入れない**（不必要なソートを避ける）。
* ウィンドウ版採用時は **ワークテーブルサイズ** に注意（`tmp_table_size`, `max_heap_table_size`）。

## 7) エッジケース備考

* 本問題は **(lat, lon) は NOT NULL** 指定。現場データで浮動小数の同値判定に不安がある場合は、**丸めて正規化**（例: 小数6桁に `ROUND` して格納）や、**都市IDキー** を別表管理に置換するのが堅実です。
* `ROUND(SUM(...), 2)` は **合計→丸め** の順序で要件どおり。行ごと丸め→合算はNG。

## 8) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Insurance 全件] --> B[集約 d: tiv_2015 ごと COUNT]
  A --> C["集約 u: (lat,lon) ごと COUNT"]
  B -->|HAVING COUNT>1| D[重複 tiv_2015 集合]
  C -->|HAVING COUNT=1| E["一意 (lat,lon) 集合"]
  D --> F[JOINで対象行だけ抽出]
  E --> F
  F --> G["SUM(tiv_2016) → ROUND(…,2)"]
  G --> H[出力 tiv_2016]
```
